# A Comprehensive Analysis of NBA Player Salaries


## Introduction

## Data Description

## Exploratory Data Analysis

## Model Building and Evaluation

## Results and Conclusion

# Author Contributions
- **Brian Fernando:** Brian worked on scraping the data. Brian also worked on feature engineering, setting up yaml file. Brian also helped with main.ipynb notebook
- **Sharona Yang:** Sharona worked on the modeling section of the analysis. Sharona also helped put together the pdf files for the MyST site and worked on the main.ipynb file.
- **Aarush Maddela:** Aarush worked on the EDA section of the project, sifting through the data to find all the trends and insights. Aarush also generated plots and figures.
- **Nixon Tan:** Nixon worked on the Makefile and running the tests. Nixon also helped created the Myst site and helped with the ai-documentation and merging PRs.
